In [1]:
import numpy as np
import random
import math

# Generation of Simulated Images

In [2]:
def generate_microscope_image(size):
  """
  Creates a binary image with a random, blob-like parasite occupying 25% or more of the area.
  """
  image = np.zeros((size, size))

  # Ensure at least 25% of pixels are black (parasite)
  parasite_area = 0.25
  min_black_pixels = 0.25 * (size**2)
  black_pixels = 0

  # Start with a random seed pixel
  x, y = random.randint(0, size-1), random.randint(0, size-1)
  image[x][y] = 1
  black_pixels += 1

  # Perform a random walk to create a connected structure
  for _ in range(int(min_black_pixels // 2)):
    # Choose a random direction (up, down, left, right)
    direction = random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)])
    new_x, new_y = x + direction[0], y + direction[1]

    # Check if new position is within image bounds and not already black
    if 0 <= new_x < size and 0 <= new_y < size and image[new_x][new_y] == 0:
      x, y = new_x, new_y
      image[x][y] = 1
      black_pixels += 1

  # Fill any small gaps within the initial structure to improve blob-likeness
  fill_queue = [(x, y)]
  while fill_queue:
    x, y = fill_queue.pop(0)
    for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
      new_x, new_y = x + dx, y + dy
      if 0 <= new_x < size and 0 <= new_y < size and image[new_x][new_y] == 0:
        image[new_x][new_y] = 1
        black_pixels += 1
        fill_queue.append((new_x, new_y))

  # Add some noise for imperfections
  for _ in range(int(size**2 * 0.01)):
    x, y = random.randint(0, size-1), random.randint(0, size-1)
    image[x][y] = 1

  # Generate RLE data for each row
  rle_data = {}
  for y in range(size):
    # Initialize variables for current run
    runs = []
    start_col = None
    end_col = None
    for x in range(size):
      if image[y, x] == 0:  # Black pixel encountered
        if start_col is None:  # Start of a new run
          start_col = x
        end_col = x  # Update end of current run
      elif start_col is not None:  # End of a black pixel run
        runs.append((start_col, end_col))  # Add run data (start, end)
        start_col = None
        end_col = None
    # Add the last run if it exists
    if start_col is not None:
      rle_data[y] = runs

  return rle_data

In [3]:
# Sparse matrix representation is used for dye sensor images using dictionary as the lit pixels will be sparse

def generate_dye_sensor_image(size, sparsity=0.001):
  """
  Creates a sparse image with random luminescence points.
  """
  image = []  # Use list for sparse representation of coordinates of lit pixels
  num_lit_pixels = int(size**2 * sparsity)

  # Generate starting point for vein
  center_x = random.randint(1, size - 2)
  center_y = random.randint(1, size - 2)

  # Perform multiple random walks to create vein-like structure for the dye image
  for _ in range(5):  # Number of random walks

    current_x = center_x
    current_y = center_y
    num_steps = random.randint(0, num_lit_pixels)
    num_lit_pixels -= num_steps

    for _ in range(num_steps):
      # Choose a random direction with some bias towards staying connected
      directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
      weight = [0.4, 0.2, 0.2, 0.2]  # Less bias towards staying connected
      direction = random.choices(directions, weights=weight)[0]
      new_x = current_x + direction[0]
      new_y = current_y + direction[1]
      # Check if new position is within parasite and not already lit
      if 0 <= new_x < size and 0 <= new_y < size and (new_x, new_y) not in image:
        # Add lit pixel to sparse matrix
        image.append((new_x, new_y))
        current_x = new_x
        current_y = new_y

  return image

# Detection of Cancer

In [6]:
def has_cancer(microscope_image, dye_image, cancer_threshold=0.1):
  """
  Checks if a parasite has cancer based on dye intensity exceeding the threshold.
  """

  # Calculate total dye intensity within parasite region
  total_dye = 0
  total_parasite_area = 0

  for row in microscope_image.keys():
    for run in microscope_image[row]:
      total_parasite_area += run[1] - run[0] + 1
      for i in range(run[0], run[1] + 1): # Check if any dye is present within each run of parasite pixels
        if (row,i) in dye_image:
          total_dye += 1

  # Determine if dye intensity exceeds cancer threshold
  return total_dye / total_parasite_area > cancer_threshold

# Optimised Cancer Detection

In [8]:
"""
Optimised Cancer Detection, instead of calculating total_dye and total_parasite_area in the same nested loop which can go upto O(n^3), where n = size of the grid (100000),
we use the fact that the num of lit pixels in the dye image will be very less and the approximate number of runs in each row of the microscope image will be very less (max 10-12),
the time complexity reduces to O(n) for calculating the total_parasite_area and O(num_of_lit_pixels) for calculating total_dye within the parasite body.
"""

def has_cancer_optimised(microscope_image, dye_image, cancer_threshold=0.1):
  """
  Checks if a parasite has cancer based on dye intensity exceeding the threshold.
  """

  # Calculate total dye intensity within parasite region
  total_dye = 0
  total_parasite_area = 0

  # Computing total_parasite_area using RLE representation of the microcope image
  for row in microscope_image.keys():
    for run in microscope_image[row]:
      total_parasite_area += run[1] - run[0] + 1

  for lit_pixel in dye_image:
    for run in microscope_image[lit_pixel[0]]:
      if(lit_pixel[1]>=run[0] and lit_pixel[1]<=run[1]): # Check if the dye coordinate lies within the parasite body
        total_dye += 1
        break
    if(total_dye/total_parasite_area > cancer_threshold): # Early stopping for efficiency
      return True

  # Determine if dye intensity exceeds cancer threshold
  return total_dye / total_parasite_area > cancer_threshold
